In [31]:
import torch

# 打印出可用的 CUDA 设备数量
print("Available CUDA devices:", torch.cuda.device_count())

Available CUDA devices: 4


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset
import torchvision
from torchvision import models
from tqdm import tqdm
import pickle

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:

import os.path as osp
from torchvision.datasets import ImageFolder


class GTSRB(ImageFolder):
    def __init__(self, train=True, transform=None, target_transform=None, root="./../../data/gtsrb"):
        root = osp.join(root, 'GTSRB', 'Final_Training')
        if not osp.exists(root):
            raise ValueError('Dataset not found at {}. Please download it from {}.'.format(
                root, 'http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset'
            ))

        # Initialize ImageFolder
        super().__init__(root=osp.join(root, 'Images'), transform=transform,
                         target_transform=target_transform)

        self.root = root
        trainning_size = len(self.samples)
        self.read_test(osp.join('./../../data/gtsrb', 'GTSRB', 'Final_Test', 'Images'))

        self.partition_to_idxs = self.get_partition_to_idxs(trainning_size)
        self.pruned_idxs = self.partition_to_idxs['train' if train else 'test']

        # Prune (self.imgs, self.samples to only include examples from the required train/test partition
        self.samples = [self.samples[i] for i in self.pruned_idxs]
        self.imgs = self.samples

        print('=> done loading {} ({}) with {} examples'.format(self.__class__.__name__, 'train' if train else 'test',
                                                                len(self.samples)))

    def read_test(self, folder):
        with open(osp.join(folder, "GT-final_test.csv")) as f:
            f.readline()  # 跳过标题行
            for line in f:
                parts = line.strip().split(";")
                # 假设最后一个字段是标签
                label = parts[-1]
                # 图像路径是第一个字段
                image = parts[0]
                path = osp.join(folder, image)
                self.samples.append((path, int(label)))
                self.targets.append(int(label))

    def get_partition_to_idxs(self, training_size):
        partition_to_idxs = {
            'train': [],
            'test': []
        }
        for i in range(training_size):
            partition_to_idxs['train'].append(i)
        for i in range(training_size, len(self.samples)):
            partition_to_idxs['test'].append(i)
        return partition_to_idxs

root = './../../data/gtsrb'  # 数据集存储的根目录]
transform_train = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010)),
        ])
transform_test = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010)),
        ])
train_set = GTSRB(train=True, transform=transform_train, root = root)
test_set = GTSRB(train=False, transform=transform_test, root = root)
batch_size = 1500 # 每个批次的样本数
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)
data_iter = iter(train_loader)
X_sub, _ = next(data_iter)
X_sub = X_sub.numpy()
print(X_sub)
print(len(X_sub))

=> done loading GTSRB (train) with 39209 examples
=> done loading GTSRB (test) with 12630 examples
[[[[-2.08013725e+00 -2.04136753e+00 -1.98321271e+00 ... -2.02198267e+00
    -2.02198267e+00 -2.04136753e+00]
   [-2.08013725e+00 -2.04136753e+00 -1.96382785e+00 ... -1.98321271e+00
    -2.00259757e+00 -2.02198267e+00]
   [-2.00259757e+00 -2.04136753e+00 -1.98321271e+00 ... -1.98321271e+00
    -2.00259757e+00 -2.00259757e+00]
   ...
   [-1.80874848e+00 -1.88628817e+00 -1.96382785e+00 ... -2.13829207e+00
    -2.13829207e+00 -2.15767694e+00]
   [-1.92505801e+00 -1.96382785e+00 -1.96382785e+00 ... -2.15767694e+00
    -2.15767694e+00 -2.15767694e+00]
   [-1.98321271e+00 -1.90567303e+00 -1.90567303e+00 ... -2.17706180e+00
    -2.17706180e+00 -2.17706180e+00]]

  [[-2.08391857e+00 -2.06425166e+00 -2.00525117e+00 ... -2.04458475e+00
    -2.08391857e+00 -2.06425166e+00]
   [-2.10358524e+00 -2.06425166e+00 -2.00525117e+00 ... -2.06425166e+00
    -2.06425166e+00 -2.06425166e+00]
   [-2.06425166e+00 

In [3]:
root = './../../data/cifar10'  # 数据集存储的根目录

# 定义数据集的变换
trainset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

testset = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2470, 0.2435, 0.2616)),
])

# 加载数据集
train_set = datasets.CIFAR10(root, train=True, download=True, transform=trainset)
test_set = datasets.CIFAR10(root, train=False, download=True, transform=testset)

# 定义 DataLoader
batch_size = 1500
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

# 获取一个批次的数据
data_iter1 = iter(train_loader)
X_sub1, _ = next(data_iter1)
X_sub1 = X_sub1.numpy()
print(X_sub1)
print(len(X_sub1))


Files already downloaded and verified
Files already downloaded and verified
[[[[ 0.34441543  0.37616903  0.63019776 ...  0.18564747  0.31266186
     0.31266186]
   [ 0.455553    0.32853863  0.5508138  ...  0.29678506  0.21740107
     0.34441543]
   [ 0.63019776  0.42379943  0.50318336 ...  0.455553    0.18564747
     0.1221403 ]
   ...
   [ 1.5828055   1.5669287   1.4716679  ...  0.4873066   0.5825674
     0.59844416]
   [ 1.5351751   1.5510519   1.5192983  ...  0.50318336  0.53493696
     0.50318336]
   [ 1.4240376   1.4081607   1.4399143  ...  0.53493696  0.455553
     0.26503146]]

  [[ 0.3227283   0.37104332  0.6287234  ...  0.20999329  0.35493833
     0.35493833]
   [ 0.45156834  0.33883333  0.5481984  ...  0.38714832  0.27441332
     0.38714832]
   [ 0.6287234   0.41935834  0.49988335 ...  0.53209335  0.12946826
     0.11336327]
   ...
   [ 1.7399687   1.7238637   1.6433387  ...  0.5643034   0.7092484
     0.6931434 ]
   [ 1.6755487   1.6594436   1.7077587  ...  0.49988335  0.548

In [4]:
# 连接数组
concat_data = np.concatenate((X_sub, X_sub1), axis=0)
#print(concat_data)
print(len(concat_data))


3000


In [5]:
import numpy as np

# 加载数据
loaded_data = np.load('../../gtsrb/X_sub3000.npy')
print("原始数据长度:", len(loaded_data))
print("原始数据形状:", loaded_data.shape)

# 获取后1500个数据
last_1500_data = loaded_data[-1500:]
print("后1500个数据的长度:", len(last_1500_data))
print("后1500个数据的形状:", last_1500_data.shape)

原始数据长度: 3000
原始数据形状: (3000, 3, 32, 32)
后1500个数据的长度: 1500
后1500个数据的形状: (1500, 3, 32, 32)


In [6]:
# 连接数组
concat_data = np.concatenate((concat_data, last_1500_data ), axis=0)
#print(concat_data)
print(len(concat_data))


4500


In [9]:
# 创建全零标签数组
labels_sub = np.zeros(concat_data.shape[0])

# 将前一半样本的标签设置为0，后一半样本的标签设置为1
#labels_sub[concat_data.shape[0] // 2:] = 1
#labels_sub[:150] = 0
#labels_sub[150:] = 1
# 将concat_data分成三个部分，并为每个部分赋予相应的标签
labels_sub[:1500] = 0  # 前1500个元素的标签为0
labels_sub[1500:3000] = 1  # 接下来的1500个元素的标签为1
labels_sub[3000:] = 2  # 最后1500个元素的标签为2
#labels_sub[4500:] = 3

# 打印标签数组的长度和0形状
print(len(labels_sub))
print(labels_sub.shape)

4500
(4500,)


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 转换为PyTorch的Tensor对象
X_tensor = torch.from_numpy(concat_data).float()
y_tensor = torch.from_numpy(labels_sub).long()


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test)
print("训练集长度:", len(X_train))
print("测试集长度:", len(X_test))
print("训练标签长度:", len(y_train))
print("测试标签长度:", len(y_test))
print(X_train.shape)
print(y_train.shape)

tensor([[[[ 1.8591e+00,  1.8591e+00,  1.8591e+00,  ...,  2.2274e+00,
            2.1480e+00,  1.6051e+00],
          [ 1.8591e+00,  1.8591e+00,  1.8591e+00,  ...,  1.7321e+00,
            9.7001e-01,  7.9536e-01],
          [ 1.8591e+00,  1.8591e+00,  1.8591e+00,  ...,  2.1638e+00,
            1.3511e+00,  1.0366e+00],
          ...,
          [ 1.8591e+00,  1.8591e+00,  1.8591e+00,  ...,  1.6686e+00,
            1.3828e+00,  1.7511e+00],
          [ 2.2591e+00,  2.2591e+00,  2.2591e+00,  ...,  2.5555e-01,
           -7.4469e-01, -4.9066e-01],
          [ 2.2591e+00,  2.2591e+00,  2.2591e+00,  ..., -7.4469e-01,
           -1.0305e+00, -9.5108e-01]],

         [[ 1.9265e+00,  1.9265e+00,  1.9265e+00,  ...,  1.8782e+00,
            2.2299e+00,  2.1010e+00],
          [ 1.9265e+00,  1.9265e+00,  1.9265e+00,  ...,  2.2460e+00,
            1.0890e+00,  7.8303e-01],
          [ 1.9265e+00,  1.9265e+00,  1.9265e+00,  ...,  2.3104e+00,
            1.4917e+00,  7.5082e-01],
          ...,
     

In [11]:
model = models.vgg16(pretrained=False)

# 获取模型的输入特征维度
#num_features = model.fc.in_features

# 修改模型的全连接层
#num_classes = 3  # 新的类别数量
#model.fc = torch.nn.Linear(num_features, num_classes)
#print(model)
# 修改模型的输出类别数量
num_classes = 3  # 假设要将类别数量修改为10
model.classifier[6] = torch.nn.Linear(4096, num_classes)

# 打印修改后的模型结构
print(model)

/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [13]:
# 定义逻辑回归模型
num_classes = 3
#model = MultiClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# 将数据和标签送入设备（如GPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
model = model.to(device)
# 创建训练数据加载器
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# 创建测试数据加载器
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [27]:
# 训练模型
num_epochs = 55
model.train()
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
      
        # 前向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 打印训练信息
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
# 保存训练参数
torch.save(model.state_dict(), './gtsrb/attackgtsrb1.pth')

Epoch [1/55], Loss: 1.0683
Epoch [2/55], Loss: 1.0895
Epoch [3/55], Loss: 1.0335
Epoch [4/55], Loss: 0.6628
Epoch [5/55], Loss: 0.3944
Epoch [6/55], Loss: 0.4668
Epoch [7/55], Loss: 0.0680
Epoch [8/55], Loss: 0.1475
Epoch [9/55], Loss: 0.0373
Epoch [10/55], Loss: 0.0268
Epoch [11/55], Loss: 0.0390
Epoch [12/55], Loss: 0.2986
Epoch [13/55], Loss: 0.0024
Epoch [14/55], Loss: 0.0445
Epoch [15/55], Loss: 0.0078
Epoch [16/55], Loss: 0.0017
Epoch [17/55], Loss: 0.1600
Epoch [18/55], Loss: 0.1266
Epoch [19/55], Loss: 0.0122
Epoch [20/55], Loss: 0.0016
Epoch [21/55], Loss: 0.0004
Epoch [22/55], Loss: 0.0553
Epoch [23/55], Loss: 0.0329
Epoch [24/55], Loss: 0.0025
Epoch [25/55], Loss: 0.0022
Epoch [26/55], Loss: 0.0012
Epoch [27/55], Loss: 0.0086
Epoch [28/55], Loss: 0.0268
Epoch [29/55], Loss: 0.0001
Epoch [30/55], Loss: 0.0003
Epoch [31/55], Loss: 0.0204
Epoch [32/55], Loss: 0.0004
Epoch [33/55], Loss: 0.0003
Epoch [34/55], Loss: 0.0004
Epoch [35/55], Loss: 0.0000
Epoch [36/55], Loss: 0.0005
E

In [14]:
# 在测试集上评估模型性能
model.load_state_dict(torch.load('./gtsrb/attackgtsrb1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
test_loss = 0.0
correct = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
#       print(outputs)
        _, predicted = torch.max(outputs, 1)
#        print(predicted)
#        print(labels)
        if predicted == labels:
            correct += 1
    test_accuracy = correct / len(test_dataset)
print(f'Test Accuracy: {test_accuracy:.4f}')


Test Accuracy: 0.9656


In [29]:
# 初始化计数器
TP = 0
FP = 0
TN = 0
FN = 0
model.load_state_dict(torch.load('./gtsrb/attackgtsrb1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
# 在测试集上评估模型性能
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)    
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        # 更新计数器
        for label, prediction in zip(labels, predicted):
            if label == 0 and prediction == 0:
                TP += 1
            elif label != 0 and prediction == 0:
                FP += 1
            elif label == 0 and prediction != 0:
                FN += 1
            elif label != 0 and prediction != 0:
                TN += 1

# 计算FPR和FNR
FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0

print(f"FPR: {FPR:.4f}")
print(f"FNR: {FNR:.4f}")


FPR: 0.0187
FNR: 0.0289


In [16]:
TP = 0  # 真阳性
FN = 0  # 假阴性
model.load_state_dict(torch.load('./gtsrb/attackgtsrb1.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)

        for label, prediction in zip(labels, predicted):
            if label == 2 and prediction == 2:
                TP += 1  # 真阳性：正确预测为类别1
            elif label == 2 and prediction != 2:
                FN += 1  # 假阴性：错误预测为非类别1

# 计算 FNR
FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
print(f"FNR for class 1: {FNR:.4f}")


FNR for class 1: 0.0102


In [30]:

import os.path as osp
from torchvision.datasets import ImageFolder


class GTSRB(ImageFolder):
    def __init__(self, train=True, transform=None, target_transform=None, root="./../../data/gtsrb"):
        root = osp.join(root, 'GTSRB', 'Final_Training')
        if not osp.exists(root):
            raise ValueError('Dataset not found at {}. Please download it from {}.'.format(
                root, 'http://benchmark.ini.rub.de/?section=gtsrb&subsection=dataset'
            ))

        # Initialize ImageFolder
        super().__init__(root=osp.join(root, 'Images'), transform=transform,
                         target_transform=target_transform)

        self.root = root
        trainning_size = len(self.samples)
        self.read_test(osp.join('./../../data/gtsrb', 'GTSRB', 'Final_Test', 'Images'))

        self.partition_to_idxs = self.get_partition_to_idxs(trainning_size)
        self.pruned_idxs = self.partition_to_idxs['train' if train else 'test']

        # Prune (self.imgs, self.samples to only include examples from the required train/test partition
        self.samples = [self.samples[i] for i in self.pruned_idxs]
        self.imgs = self.samples

        print('=> done loading {} ({}) with {} examples'.format(self.__class__.__name__, 'train' if train else 'test',
                                                                len(self.samples)))

    def read_test(self, folder):
        with open(osp.join(folder, "GT-final_test.csv")) as f:
            f.readline()  # 跳过标题行
            for line in f:
                parts = line.strip().split(";")
                # 假设最后一个字段是标签
                label = parts[-1]
                # 图像路径是第一个字段
                image = parts[0]
                path = osp.join(folder, image)
                self.samples.append((path, int(label)))
                self.targets.append(int(label))

    def get_partition_to_idxs(self, training_size):
        partition_to_idxs = {
            'train': [],
            'test': []
        }
        for i in range(training_size):
            partition_to_idxs['train'].append(i)
        for i in range(training_size, len(self.samples)):
            partition_to_idxs['test'].append(i)
        return partition_to_idxs
batch_size = 1
root = './../../data/gtsrb'  # 数据集存储的根目录]
transform_train = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010)),
        ])
transform_test = transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 std=(0.2023, 0.1994, 0.2010)),
        ])
train_set = GTSRB(train=True, transform=transform_train, root = root)
test_set = GTSRB(train=False, transform=transform_test, root = root)
batch_size = 1500 # 每个批次的样本数
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

model = models.vgg16(pretrained=False)

# 修改模型的全连接层
num_classes = 3  # 新的类别数量
model.classifier[6] = torch.nn.Linear(4096, num_classes)
model.load_state_dict(torch.load('./gtsrb/attackgtsrb1.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

=> done loading GTSRB (train) with 39209 examples
=> done loading GTSRB (test) with 12630 examples


/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/CGAN/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Validation Progress: 100%|████████████████████████████████████| 12630/12630 [00:31<00:00, 395.81it/s]

input: 12630
IN: 11886
Knockoff Data: 290
JBDA Data: 454


In [6]:
root = './../../data/cifar10'
train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 #std=(0.2023, 0.1994, 0.2010)),
                                 std=(0.2470, 0.2435, 0.2616)),
        ])
test_transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                                 #std=(0.2023, 0.1994, 0.2010)),
                                 std=(0.2470, 0.2435, 0.2616)),
        ])

data_transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225]),
        ])
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

batch_size = 1 # 每个批次的样本数
train_set = datasets.CIFAR10(root, train=True, download=True, transform=train_transform)
test_set = datasets.CIFAR10(root, train=False, download=True, transform=test_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, shuffle=False)

IN_count = 0
JBDA_count = 0
Knockoff_count = 0
input_count = 0

model = models.vgg16(pretrained=False)

# 修改模型的全连接层
num_classes = 3  # 新的类别数量
model.classifier[6] = torch.nn.Linear(4096, num_classes)
model.load_state_dict(torch.load('./gtsrb/attackgtsrb.pth'))
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs, _ in tqdm(test_loader, desc="Validation Progress"):
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            IN_count += 1
        elif predicted == 1 :
            Knockoff_count += 1
        elif predicted == 2:
            JBDA_count += 1
    print(f"input: {input_count:}")
    print(f"IN: {IN_count:}")
    print(f"Knockoff Data: {Knockoff_count:}")
    print(f"JBDA Data: {JBDA_count:}")

Files already downloaded and verified
Files already downloaded and verified


Validation Progress: 100%|████████████████████████████████████| 10000/10000 [00:23<00:00, 431.82it/s]

input: 10000
IN: 153
Knockoff Data: 9843
JBDA Data: 4


In [ ]:
# 加载.npy文件
loaded_data = np.load('../../mnist/X_sub_4.npy')
print(len(loaded_data))
print(loaded_data.shape)# 查看数组内容
#print(loaded_data)

In [50]:
# 创建全零标签数组
labels_sub = np.zeros(loaded_data.shape[0])

# 将前一半样本的标签设置为0，后一半样本的标签设置为1
#labels_sub[loaded_data.shape[0] // 2:] = 1
labels_sub[:150] = 0
labels_sub[150:] = 1
print(labels_sub)
print(len(labels_sub))

[0. 0. 0. ... 1. 1. 1.]
4800


In [51]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# 转换为PyTorch的Tensor对象
X_tensor = torch.from_numpy(loaded_data).float()
y_tensor = torch.from_numpy(labels_sub).long()


# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)
print(X_train, X_test, y_train, y_test)
print("训练集长度:", len(X_train))
print("测试集长度:", len(X_test))
print("训练标签长度:", len(y_train))
print("测试标签长度:", len(y_test))

tensor([[[[-0.0242, -0.4242, -0.4242,  ..., -0.8242, -0.8242, -0.6242],
          [-0.8242, -0.8242, -0.4242,  ..., -0.4242, -0.0242, -0.6242],
          [-0.4242, -0.8242, -0.8242,  ..., -0.4242, -0.0242, -0.6242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.8242, -0.6242],
          [-0.0242, -0.4242, -0.8242,  ..., -0.8242, -0.8242, -0.6242],
          [-0.2242, -0.2242, -0.6242,  ..., -0.6242, -0.6242, -0.6242]]],


        [[[-0.2242, -0.2242, -0.2242,  ..., -0.2242, -1.0242, -0.4242],
          [-0.6242,  0.1758, -1.0242,  ..., -0.6242, -1.0242, -0.0242],
          [-0.6242,  0.1758, -0.2242,  ..., -1.0242, -1.0242, -0.6242],
          ...,
          [-0.6242, -1.0242,  0.1758,  ..., -0.2242, -0.6242, -0.4242],
          [-0.2242, -0.6242, -1.0242,  ..., -0.2242, -0.6242, -0.8242],
          [-0.2242, -0.2242,  0.1758,  ..., -0.4242, -0.0242, -0.6242]]],


        [[[-0.0242, -0.0242, -0.0242,  ..., -0.8242,  0.3758, -0.2242],
          [ 0.3758, -0.424

In [68]:
# 定义逻辑回归模型
model = MultiClassifier(num_classes)
model.load_state_dict(torch.load('attackmnist.pth'))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
# 创建测试数据加载器
test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [69]:
# 在测试集上评估模型性能
model.eval()
with torch.no_grad():
    for inputs,_ in test_loader:
        input_count += 1
        inputs = inputs.to(device)
        outputs = model(inputs)
#        print(outputs)
        _, predicted = torch.max(outputs, 1)
        if predicted == 0 :
            MNIST_count += 1
        elif predicted == 1 :
            MNIST1_count += 1
        else :
            EMNIST_count += 1
#        print(labels)
    print(f"input: {input_count:}")
    print(f"MNIST: {MNIST_count:}")
    print(f"EMNIST: {EMNIST_count:}")
    print(f"MNIST1: {MNIST1_count:}")

input: 21522
MNIST: 10271
EMNIST: 172
MNIST1: 11077
